<a href="https://colab.research.google.com/github/khalil753/Bio-Project/blob/master/Deterministic2Bayesian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mounting libraries and defining global variables**

In [0]:
import fastai
from fastai.vision import *
from fastai.widgets import *

import numpy as np
import pandas as pd

from pathlib import Path
import os
from os import listdir

import matplotlib as mpl
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import tensor

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
sns.set()

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
path = Path(r"/content/gdrive/My Drive/Mestría en Física de Sistemas Complejos/Tercer semestre/Inference in Biological Systems/Project")
path_data = path/'data'
path_clean_losses = path_data/"Clean Losses"
path_losses = path_data/"Losses"
classes = ['healthy', 'tumor', 'injury']

# **Defining Bayesian Dropout modules**

In [0]:
class BYDropout(nn.Module):
  """Custom Dropout module to be used for MC Dropout"""

  def __init__(self, p:float, activate=True):
    super().__init__()
    self.activate = activate
    self.p = p

  def forward(self, x):
    return F.dropout(x, self.p, training=self.training or self.activate)

class BYDropout2d(nn.Module):
  """Custom Dropout2d module to be used for MC Dropout"""

  def __init__(self, p:float, activate=True):
    super().__init__()
    self.activate = activate
    self.p = p

  def forward(self, x):
    return F.dropout2d(x, self.p, training=self.training or self.activate)

def set_bayes(m, activate:bool=True):
  if isinstance(m, BYDropout) or isinstance(m, BYDropout2d):
      print(f"Current active : {c.activate}")
      print(f"Switching to : {activate}")
      m.activate = activate

def set_p(module, p):
  if isinstance(module, BYDropout) or isinstance(module, BYDropout2d):
    module.p = p

def Dropout2BY(model:nn.Module):
  """
  Convert modules of type Dropout to BYDropout inside the model
  """
  for child_name, child in model.named_children():
    if isinstance(child, nn.Dropout):
      # First we grab p from the child
      p = child.p         
      new_layer = BYDropout(p)
      setattr(model, child_name, new_layer)
    if isinstance(child, nn.Dropout2d):
      # First we grab p from the child
      p = child.p         
      new_layer = BYDropout2d(p)
      setattr(model, child_name, new_layer)
    else:
      Dropout2BY(child)

In [0]:
nn.Module.to_BY = lambda

In [0]:
loading_data = False
make_bayes = False
making_preds = False
caculating_uncertanties = False

### Loading the model

In [0]:
if making_preds:
  learn = load_learner(path_data)
  m = learn.model

In [0]:
if loading_data:
  np.random.seed(42)
  path_data = path/'data'
  tfms = get_transforms(do_flip=False)
  data = ImageDataBunch.from_folder(path_data, ds_tfms=tfms, size=32,  
                                    train="train",
                                    valid="val",
                                    classes=classes, bs=200).normalize(imagenet_stats)

In [0]:
if making_preds:
  learn.data = data                                       

### Adding the Dropout layers

In [0]:
if make_bayes:
  BasicBlock = type(learn.model[0][5][0])
  def addDPs(m):
    m_dp = []
    try: iter(m)
    except TypeError: return m
    for mod in m:
      if not isinstance(mod, BasicBlock):
        mod = addDPs(mod)
      m_dp.append(mod)
      if isinstance(mod, BasicBlock): m_dp.append(BYDropout2d(p=0.5))
    
    return nn.Sequential(*m_dp) 

In [0]:
if make_bayes:
  a = list(m[0])
  a.insert(1, BYDropout2d(p=0.5))
  m[0] = nn.Sequential(*a)
  m = nn.Sequential(*m)

In [0]:
if make_bayes:
  m = addDPs(m)

In [0]:
if make_bayes:
  learn.model = m
  Dropout2BY(learn.model)

### Making predictions

In [0]:
if making_preds:
  def get_n_scores(learn, ds_type=DatasetType.Train, n_pred=10):
    bs =  learn.data.batch_size
    n_data = len(learn.data.train_ds)
    all_scores = torch.zeros((n_data - n_data % bs,0,3))    #
    for i in range(n_pred):
      scores, target = get_preds(learn.model, learn.data.train_dl)
      all_scores = torch.cat((all_scores, scores.unsqueeze(1)), dim=1)
      print(f"Predictions done: {i}/{n_preds}")
    return all_scores, target

In [0]:
if making_preds:
  set_bayes(learn.model, activate=True)
  learn.model.apply(lambda x: set_p(x, 0.1))
  train_scores, train_target = get_n_scores(learn, n_pred=20)

In [0]:
if making_preds:
  prev_train_scores = torch.load(path_data/'train_scores.pt') 
  train_scores = torch.cat((prev_train_scores, train_scores), 2)
  torch.save(train_scores, path_data/'train_scores.pt') 

In [0]:
if caculating_uncertanties:
  train_scores = torch.load(path_data/'train_scores.pt') 
  probs = train_scores.softmax(1)
  probs.shape

### Calculating the Image Uncertanties

In [0]:
if caculating_uncertanties:
  def uncertanties(probs):
    uncertanties = torch.zeros((12200,))
    for s, sample in enumerate(probs):
      T = len(sample)
      for p in sample:
        mean = p.mean()
        ale = (p.diag() - p.unsqueeze(-1) * p)
        x = p - mean
        epi = (x.unsqueeze(-1) * x)
        uncertanties[s] = (ale + epi).sum()/T
    return uncertanties

In [0]:
if caculating_uncertanties:
  train_scores = train_scores.view((-1,3,21))

In [0]:
if caculating_uncertanties:
  unc = torch.zeros((len(train_scores),))
  for i, score in enumerate(train_scores):
    unc[i] = tensor(calc_uncertainity_softmax(score)[0])#.sum()
  unc[:5]

In [0]:
# if caculating_uncertanties:
#   n_plots = 1
#   fig, ax = plt.subplots(1, n_plots, figsize = (6*n_plots,4))

#   ax.get_xaxis().tick_bottom()
#   ax.get_yaxis().tick_left()
#   ax.spines["top"].set_visible(False)
#   ax.spines["right"].set_visible(False)
#   # ax.set_xlabel("Variance", fontsize=16)
#   ax.set_title("Variance", fontsize=18)

#   ax.hist(unc, bins=20, color="#3F5D7D", rwidth=.95)  
#   plt.tight_layout()
#   plt.show()
#   #fig.savefig(path + "Final Report/typefig/Amount.pgf")

### Removing high uncertanty images from training

In [0]:
if caculating_uncertanties:
  low_S_idxs = torch.arange(len(train_S))[train_S <= 1.25]
  len(low_S_idxs), len(train_S)